# Working Samples

## Include files

In [1]:
workspace = str('C:/Users/razkey/git/HybridMem-Scheduler')

In [72]:
includes = ['request','trafficGen','page','addressSpace','scheduler','pageSelector','profile','performanceModel']
files=[]
for component in includes:
    file = workspace+'/MemProfiling/'+component
    print(file)
    %run $file

C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/request
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/trafficGen
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/page
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/addressSpace
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/scheduler
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/pageSelector
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/profile
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/performanceModel


In [73]:
lstmIncludes = ['LSTM_input','lstm']
files=[]
for component in lstmIncludes:
    file = workspace+'/MemProfiling/LSTM/'+component
    print(file)
    %run $file

C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/LSTM/LSTM_input
C:/Users/razkey/git/HybridMem-Scheduler/MemProfiling/LSTM/lstm


## Sample 1

### Load input trace

In [4]:
#Available benchmarks
benchmarks= ["backprop_100k","streamcluster","lud_2048","blackscholes","bodytrack","bplustree","hotspot_1024","kmeans_80k"]

#Choose trace file
trace_file=workspace+'/traces/'+benchmarks[0]


In [5]:
'''
Read Trace, Convert Input raw txt to python objects with useful attributes
'''

prof = Prof(trace_file)
prof.init()

### Run History Scheduler

In [6]:
'''
 GET Per Page Access Counts using a History Page Scheduler
 necessary step for Page Selector

 epochs=100 , input file is broken into 100 epochs
 policy=history , history scheduler is used
 cap_ratio=1/8 , 1/8 fit in DRAM , 7/8 in NVM 
'''

epochs = 100
num_reqs = len(prof.traffic.req_sequence)//epochs
sim = PerformanceModel(prof,'TEST-HMS',policy='history',cap_ratio=1/8,num_reqs=len(prof.traffic.req_sequence)//epochs)
sim.init()
sim.run()

[Scheduler] Initialization done for policy = history periods = 101 reqs per period = 5892 cap ratio = 0.125


### Run Page Selector
We now run Page Selector (afte running the History Scheduler) to find which pages are heavily misplaced and then choose the ones we want for RNN training

In [7]:
'''
GET Misplaced pages eligible for RNN training 
'''
page_selector = PageSelector(prof, 'TEST-HMS', '0.25', num_reqs)
pages_misplaced = page_selector.get_misplaced_pages_sim()
pages_ordered = page_selector.get_ordered_pages(pages_misplaced)

In [48]:
'''
How many RNNs to deploy
'''
page_id_x = pages_ordered[:1] #Chose one as a sample

1756


### RNN training
Now we train RNN for the selected pages. I only train one RNN for the most heavily misplaced page

#### RNN Input

In [44]:
### Make the RNN input
# Step 1: take the page access count across periods
cnts_x = prof.hmem.page_list[page_id_x[0]].oracle_counts_binned_ep
input = LSTM_input(cnts_x)
  
# Step 2: Roll a window of history length over the periods
history_length = 20 # periods
input.timeseries_to_history_seq(history_length)
  
# Step 3: Split into training, validation and test samples througout the epochs
input.split_data(0.15)
  
# Step 4: Bring input into format for RNN training
num_classes = max(set(cnts_x)) + 1
input.to_categor(num_classes)

#### Model Init and train

In [45]:
### Initiliaze model
model = LSTM_model(input)

#initialize weights for custom loss function
weights=[]
for i in range(num_classes):
    if i==0:
        weights.append(30)
    else:
        weights.append(1)
model.weights=weights

In [46]:
### Create model
#Setup weights for loss function
model.weights=weights

###arg1(layers)=256 , learning rate(arg2)=0.001, history_length(arg3)=20, num_classes(arg4), weights(arg5) weights for every class
model.create(layers=256,learning_rate=0.0001, dropout=0,history_length=history_length,num_classes=num_classes)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 20, 256)           336896    
                                                                 
 lstm_12 (LSTM)              (None, 256)               525312    
                                                                 
 dense_5 (Dense)             (None, 72)                18504     
                                                                 
Total params: 880,712
Trainable params: 880,712
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\razkey\AppData\Roaming\Python\Python37\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
#Train the model
model.train()

#### Model save and load

In [61]:
#Save the model

#model is an object of type LSTM_model which contains a keras.sequential model
print(page_id_x[0])
model.model.save(workspace+'MemProfiling/models/'+'model'+str(page_id_x[0])+'.h5')

1756


In [64]:
#Load model
import tensorflow as tf
from tensorflow import keras

loaded_model = tf.keras.models.load_model(workspace+'/models/'+'model'+str(page_id_x[0])+'.h5')

In [65]:
'''
#How to use a trained model for predictions 

import random
cnts_x = prof.hmem.page_list[page_id_x[0]].oracle_counts_binned_ep
#num_classes=72
#history_length=20


accessCounts=[]
for x in range(0,21):
    accessCounts.append(random.randint(0,71))
print(len(accessCounts))
sampleinput = LSTM_input(accessCounts)

sampleinput.timeseries_to_history_seq(20)
sampleinput.split_data(1)
sampleinput.to_categor(72)



predictY = loaded_model.predict(sampleinput.trainX_categor)
predictY1 = np.array([np.argmax(x) for x in predictY])
#print(predictY)
print(predictY1)
'''

21
[0]


### Utilize Trained Models

In [68]:
# First load all the models 
# I only have one so I load only one it's for page 1756
loadedModels=[]

loaded_model = tf.keras.models.load_model(workspace+'/models/'+'model'+str(page_id_x[0])+'.h5')
#loadedModels.append(loaded_model)

#Create a dictionary with (page_id:model)
dic={}
dic[1756]=loaded_model

In [74]:
#Now assuming we have a dictionary with all the trained models + the page id

# Relaod the trace
trace_file=workspace+'/traces/backprop_100k'
prof = Prof(trace_file)
prof.init()
epochs = 100
num_reqs=len(prof.traffic.req_sequence)//epochs

sim_hybrid = PerformanceModel(prof,'Hybrid-HMS','hybrid',1/8,num_reqs=num_reqs)
sim_hybrid.init()
sim_hybrid.init_hybrid(dic)
sim_hybrid.run()

[Scheduler] Initialization done for policy = hybrid periods = 101 reqs per period = 5892 cap ratio = 0.125


In [76]:
#GET INFO FOR EVERY PAGE
page_list = sim_hybrid.profile.hmem.page_list

In [82]:
for x in page_list:
    print(vars(x))
    break

{'id': 0, 'req_ids': [0, 2, 4, 9, 10, 24, 25, 252, 1053, 1274, 1275, 3044, 3045], 'pc_ids': [], 'address': '140065108709376', 'reuse_dist': array([   2,    2,    5,    1,   14,    1,  227,  801,  221,    1, 1769,
          1]), 'misplacements': 0, 'misplacementsPeriods': [], 'writesPerPeriod': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'readsPerPeriod': array([13.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  